In [99]:
from bs4 import BeautifulSoup #parse html
import requests #html requests
import pandas as pd #data manipulation
import numpy as np #array manip
from transformers import AutoTokenizer, AutoModelForSequenceClassification #convert to tokens, model for training
import torch #big data
import re #regex: string matching

In [101]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment') #downloading hugging face transformer
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [103]:
#test the model
token=tokenizer.encode('it was okay',return_tensors='pt') #pt for pytorch tensor return values
token[0] #multidimensional array, so first row only

tensor([  101, 10197, 10140, 44810, 10158,   102])

In [105]:
tokenizer.decode(token[0]) #decode and get the message

'[CLS] it was okay [SEP]'

In [107]:
 res=model(token) #fit the tokens to the model

In [109]:
res

SequenceClassifierOutput(loss=None, logits=tensor([[-1.6243,  0.8367,  2.7918,  0.3680, -2.0830]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [111]:
res.logits #get probabilities

tensor([[-1.6243,  0.8367,  2.7918,  0.3680, -2.0830]],
       grad_fn=<AddmmBackward0>)

In [113]:
int(torch.argmax(res.logits))+1 #get highest sentiment position, rating, add 1 since it starts from there

3

# Reviews

In [116]:
from datetime import datetime
def scraper(url,results_df):
    
    myProxy = {
            "http"  : "http://10.120.118.49:8080",
            "https"  : "https://10.120.118.49:8080"
            }
    headers={'User-agent' : 'Mozilla/5.0'}
    colsList = ['Aircraft', 'Type Of Traveller', 'Seat Type', 'Route', 'Date Flown', 
            'Seat Comfort', 'Cabin Staff Service', 'Food & Beverages', 
            'Inflight Entertainment', 'Ground Service', 'Value For Money,', 'Recommended'] 
    
    page1 = requests.get(url, headers=headers)
    soup1 = BeautifulSoup(page1.text, 'html.parser')
    reviews = soup1.findAll('div', attrs={'class': "body"})
    for review in reviews:
        for span in review.find_all("span", {'class':'star fill'}): 
            span.decompose()
        
        title=review.find('h2', attrs={'class': "text_header"})
        if title is not None:
            title_of_review = title.text.strip()
        else:
            title_of_review = ''
        
        details=review.find('h3', attrs={'class': "text_sub_header userStatusWrapper"})
        if details is not None:
            details_about = details.text.strip()
        else:
            details_about = ''
        
        texto=review.find('div', attrs={'class': "text_content"})
        if texto is not None:
            review_text = texto.text.strip('✅ Verified Review |').strip('\r\n\r\n')
        else:
            review_text = ''

        
        temp_df_alpha = pd.DataFrame([[title_of_review, details_about, review_text]], columns=['Title','Details','Text'])
        temp_df_beta = pd.read_html(str(review))[0].T
        new_header = list(temp_df_beta.iloc[0])
        temp_df_beta = temp_df_beta[1:]
        temp_df_beta.columns = new_header
        cols = [ col for col in new_header if col in colsList]
        temp_df_beta = temp_df_beta[cols]
        
        temp_df = temp_df_alpha.merge(temp_df_beta.reset_index(drop=True), how='left', left_index=True, right_index=True)
        rateCols = ['Seat Comfort', 'Cabin Staff Service', 'Food & Beverages', 
            'Inflight Entertainment', 'Ground Service', 'Value For Money']
        
        for col in rateCols:
            if col in temp_df.columns:
                try:    
                    temp_df[col] = int(temp_df.iloc[0][col][0]) - 1
                except:
                    temp_df[col] = 5

        results_df = pd.concat([results_df, temp_df], ignore_index=True)

        
            #user
            #reviewer_name = review.find("span", itemprop="name") #matches tags in html file
            #name.append(reviewer_name.text.strip() if reviewer_name else "N/A")
        
        # Extract review rating value
        #rating=[]
        #for review in soup.find_all("span", itemprop="ratingValue"):
            #rating.append(review.get_text(strip=True))


    #results_df['User Name']=pd.Series(name)
    #results_df['Location']=pd.Series(loc)
    #results_df['Rating / 10']=pd.Series(rating)
    results_df
    #results_df.drop(columns=['Details'],inplace=True)
    #print(results_df)


    
    return results_df


In [118]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

results_df=pd.DataFrame()
temp_df=pd.DataFrame()

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    results_df=scraper(url,results_df) #global variable, so it affects overall
    #results_df = pd.concat([results_df, temp_df], ignore_index=True)
    
    

results_df
    

Scraping page 1


C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be 

Scraping page 2


C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be 

Scraping page 3


...

Scraping page 9


C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be 

Scraping page 10


C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df_beta = pd.read_html(str(review))[0].T
C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1417252256.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be 

,Title,Details,Text,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Recommended
0,"""disagreeable flight attendant""",10 reviews\n\n\n\nJ Meares (United Kingdom) 19...,Trip Verified | Flight mainly let down by a ...,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,5.0,4.0,yes
1,"""flight was delayed nearly 1 hour""",Luis De Jesus (United States) 16th March 2025,Trip Verified | Another awful experience by ...,NaN,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,NaN,1.0,no
2,"""full of attitude to me""",Luis De Jesus (United States) 16th March 2025,"Trip Verified | The service was rude, full o...",NaN,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,NaN,5.0,no
3,"""service was outrageous""",Luis De Jesus (United States) 16th March 2025,Trip Verified | This flight was a joke. Ther...,NaN,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,NaN,1.0,no
4,"""managed to get everything right""",38 reviews\n\n\n\nPaul Mercer (United Kingdom)...,Trip Verified | This time British Airways ma...,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,4.0,5.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"""welcome wasn’t especially friendly""",K Pellberg (Sweden) 10th September 2019,Trip Verified | Mahe to Stockholm via London. ...,Boeing 787 / A320,Solo Leisure,Business Class,Mahe to Stockholm via London,September 2019,1.0,2.0,2.0,4.0,1.0,no
996,"""been upgraded to Club World""",John Allan (United Kingdom) 8th September 2019,Trip Verified | Glasgow to Tampa via Gatwick....,A319 / Boeing777,Couple Leisure,Business Class,Glasgow to Tampa via Gatwick,September 2019,4.0,5.0,3.0,3.0,5.0,yes
997,"""standards on BA have plummeted""",S Bailey (United Kingdom) 7th September 2019,Not Verified | Orlando to Gatwick. Travelling...,Boeing 777-200,Family Leisure,Economy Class,Orlando to Gatwick,September 2019,2.0,3.0,1.0,4.0,3.0,no
998,"""Good value flight""",M Leyton (United Arab Emirates) 6th September ...,Trip Verified | London to Abu Dhabi. This is ...,Boeing 787,Family Leisure,Economy Class,London to Abu Dhabi,August 2019,3.0,4.0,5.0,5.0,4.0,yes


In [120]:
results_df['Details']=results_df['Details'].str.replace(r'reviews\n\n\n\n', ' ', regex=True) #assign only to the columns not df
results_df['Details']=results_df['Details'].str.replace(r'United Kingdom', 'United-Kingdom', regex=True) #all matching conditions found for regex
results_df['Details']=results_df['Details'].str.replace(r'United States', 'United-States', regex=True)
results_df['Details']=results_df['Details'].str.replace(r'South Africa', 'South-Africa', regex=True)
results_df['Details']=results_df['Details'].str.replace(r'New Zealand', 'New-Zealand', regex=True)
results_df['Details']=results_df['Details'].str.replace(r"\(", "| ".format, regex=True) #use \ as escape character to avoid parentheses errors as it has a value, .format to make it as text format
results_df['Details']=results_df['Details'].str.replace(r"\)", " |".format, regex=True)


In [122]:
results_df

,Title,Details,Text,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Recommended
0,"""disagreeable flight attendant""",10 J Meares | United-Kingdom | 19th March 2025,Trip Verified | Flight mainly let down by a ...,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,5.0,4.0,yes
1,"""flight was delayed nearly 1 hour""",Luis De Jesus | United-States | 16th March 2025,Trip Verified | Another awful experience by ...,NaN,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,NaN,1.0,no
2,"""full of attitude to me""",Luis De Jesus | United-States | 16th March 2025,"Trip Verified | The service was rude, full o...",NaN,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,NaN,5.0,no
3,"""service was outrageous""",Luis De Jesus | United-States | 16th March 2025,Trip Verified | This flight was a joke. Ther...,NaN,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,NaN,1.0,no
4,"""managed to get everything right""",38 Paul Mercer | United-Kingdom | 7th March 2025,Trip Verified | This time British Airways ma...,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,4.0,5.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"""welcome wasn’t especially friendly""",K Pellberg | Sweden | 10th September 2019,Trip Verified | Mahe to Stockholm via London. ...,Boeing 787 / A320,Solo Leisure,Business Class,Mahe to Stockholm via London,September 2019,1.0,2.0,2.0,4.0,1.0,no
996,"""been upgraded to Club World""",John Allan | United-Kingdom | 8th September 2019,Trip Verified | Glasgow to Tampa via Gatwick....,A319 / Boeing777,Couple Leisure,Business Class,Glasgow to Tampa via Gatwick,September 2019,4.0,5.0,3.0,3.0,5.0,yes
997,"""standards on BA have plummeted""",S Bailey | United-Kingdom | 7th September 2019,Not Verified | Orlando to Gatwick. Travelling...,Boeing 777-200,Family Leisure,Economy Class,Orlando to Gatwick,September 2019,2.0,3.0,1.0,4.0,3.0,no
998,"""Good value flight""",M Leyton | United Arab Emirates | 6th Septembe...,Trip Verified | London to Abu Dhabi. This is ...,Boeing 787,Family Leisure,Economy Class,London to Abu Dhabi,August 2019,3.0,4.0,5.0,5.0,4.0,yes


In [124]:
results_df
results_df=results_df.drop_duplicates() #drop duplicate rows
results_df.to_csv("british_airways.csv", index=False)
print("Scraping completed and data saved to british_airways_reviews.csv")

Scraping completed and data saved to british_airways_reviews.csv


In [138]:
#results_df[['Verified', 'Review']] = results_df['Text'].str.split('|', expand=True) #to split text to columns
#results_df[['From', 'To']]=results_df['Route'].str.split('to',expand=True)
results_df
#results_df.drop(columns=['Text'],inplace=True)
#results_df.to_csv("british_airways-2.csv", index=False)

,Title,Details,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Recommended,Verified,Review
0,"""disagreeable flight attendant""",10 J Meares | United-Kingdom | 19th March 2025,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,5.0,4.0,yes,Trip Verified,Flight mainly let down by a disagreeable fl...
1,"""flight was delayed nearly 1 hour""",Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,NaN,1.0,no,Trip Verified,Another awful experience by British Airways...
2,"""full of attitude to me""",Luis De Jesus | United-States | 16th March 2025,NaN,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,NaN,5.0,no,Trip Verified,"The service was rude, full of attitude to m..."
3,"""service was outrageous""",Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,NaN,1.0,no,Trip Verified,This flight was a joke. There was four peop...
4,"""managed to get everything right""",38 Paul Mercer | United-Kingdom | 7th March 2025,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,4.0,5.0,yes,Trip Verified,This time British Airways managed to get ev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"""welcome wasn’t especially friendly""",K Pellberg | Sweden | 10th September 2019,Boeing 787 / A320,Solo Leisure,Business Class,Mahe to Stockholm via London,September 2019,1.0,2.0,2.0,4.0,1.0,no,Trip Verified,Mahe to Stockholm via London. Checked in onli...
996,"""been upgraded to Club World""",John Allan | United-Kingdom | 8th September 2019,A319 / Boeing777,Couple Leisure,Business Class,Glasgow to Tampa via Gatwick,September 2019,4.0,5.0,3.0,3.0,5.0,yes,Trip Verified,Glasgow to Tampa via Gatwick. Early morning ...
997,"""standards on BA have plummeted""",S Bailey | United-Kingdom | 7th September 2019,Boeing 777-200,Family Leisure,Economy Class,Orlando to Gatwick,September 2019,2.0,3.0,1.0,4.0,3.0,no,Not Verified,Orlando to Gatwick. Travelling home was extr...
998,"""Good value flight""",M Leyton | United Arab Emirates | 6th Septembe...,Boeing 787,Family Leisure,Economy Class,London to Abu Dhabi,August 2019,3.0,4.0,5.0,5.0,4.0,yes,Trip Verified,London to Abu Dhabi. This is the daytime fli...


In [ ]:
def sentiment(review):
    token=tokenizer.encode(review,return_tensors='pt')
    print(token)
    res=model(token)
    return(int(torch.argmax(res.logits))+1)


results_df['Sentiment']=results_df['Review'].apply(lambda x:sentiment(x[:512]),print("loop complete")) #max no. of tokens model can take, decides by first 512 tokens




loop complete
tensor([[  101, 17067, 20886, 12421, 12090, 10151,   143, 31021, 37910, 11353,
         11522, 17067, 34085, 11539,   119, 10181, 60512, 10114, 24792, 10104,
         10160, 17133, 11630,   118, 12050,   119, 11574, 24792, 10262, 16642,
         10140, 10119, 87509, 50524, 10131, 10171, 12314, 15724, 10110, 23602,
         18634, 10139, 13374, 35126, 10107, 10297, 10407, 10114, 41550, 10148,
         15021, 11222, 27422, 29026, 10342, 11050,   118,   118, 11698, 38101,
           119, 70882, 10140, 12050,   119, 20997, 11416, 10140, 42700, 10171,
         43871, 60512, 10107,   119, 16749, 10140, 12050,   113, 10103, 10246,
         75289, 10682, 13020,   114,   119, 10103, 61802, 28282, 11856, 10103,
         95044, 12705, 29967, 10297, 10197, 10140, 25561, 10462, 11522,   118,
           118, 12314, 49706, 81787, 14666, 12977, 14048,   119, 21013,   143,
         88764, 27887, 11199, 11416, 10359, 13430, 11061, 19135, 49657, 10110,
         10103,   146,   102]])
tensor

C:\Users\saniy\AppData\Local\Temp\ipykernel_42908\1612669164.py:8: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  results_df['Sentiment']=results_df['Review'].apply(lambda x:sentiment(x[:512]),print("loop complete")) #max no. of tokens model can take, decides by first 512 tokens


tensor([[  101, 10103, 11416, 10140, 60295,   117, 12851, 10108, 44089, 10114,
         10525,   117, 10103, 15225, 10127, 86430, 11416, 10163,   117, 10103,
         57079, 10770, 10103, 17067, 10140, 13362, 35665, 10390, 10104, 19772,
         10103, 59868, 10870,   119,   102]])
tensor([[  101, 10372, 17067, 10140,   143, 87545,   119, 10768, 10140, 11443,
         11227, 10104, 12425, 12728, 11371, 51875, 10110, 11153, 23165,   119,
         10103, 11416, 10140, 23078, 10592, 14200,   119, 10103, 19607, 10140,
         14666, 79124, 19341, 54251, 10285, 10171, 10103, 15257, 10110, 10982,
         10572, 10140, 17010, 10139, 10103, 11111, 11856,   117, 16209, 10108,
         74838, 20726, 10104, 10103, 57079,   119,   102]])
tensor([[  101, 10372, 10573, 11289, 29480, 22399, 10114, 13168, 23225, 12873,
           119, 10103, 16993, 10108, 10103, 71327, 10140, 23065, 13356, 10110,
         10103, 88390, 10140, 23021,   119, 75625, 10140, 48093, 10110, 10103,
         26471, 10342, 66

In [75]:
'''for i in range(1000):
    results_df['Sentiment'][i]=sentiment(results_df['Review'][i][:512]) #can use double dimension to specify length of string inside for token match as max no. of tokens model can take, decides by first 512 tokens
    print(results_df['Sentiment'][i],"|", results_df['Review'][i])
    print(f"looping {i}")'''


'for i in range(1000):\n    results_df[\'Sentiment\'][i]=sentiment(results_df[\'Review\'][i][:512]) #can use double dimension to specify length of string inside for token match as max no. of tokens model can take, decides by first 512 tokens\n    print(results_df[\'Sentiment\'][i],"|", results_df[\'Review\'][i])\n    print(f"looping {i}")'

In [174]:
#print(results_df['Sentiment'])
results_df.head(20)
results_df
results_df['Title']=results_df['Title'].str.replace(r'\"', ' ', regex=True)
#


In [176]:
results_df

,Title,Details,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Recommended,Verified,Review,Sentiment,From,To,Via
0,disagreeable flight attendant,10 J Meares | United-Kingdom | 19th March 2025,Boeing 787,Solo Leisure,Business Class,Cape Town | London,March 2025,5.0,2.0,5.0,5.0,4.0,yes,Trip Verified,Flight mainly let down by a disagreeable fl...,2,10 J Meares,United-Kingdom,19th March 2025
1,flight was delayed nearly 1 hour,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,London | Geneva,December 2024,1.0,1.0,NaN,NaN,1.0,no,Trip Verified,Another awful experience by British Airways...,1,Luis De Jesus,United-States,16th March 2025
2,full of attitude to me,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Premium Economy,London | New York,January 2025,1.0,1.0,1.0,NaN,5.0,no,Trip Verified,"The service was rude, full of attitude to m...",2,Luis De Jesus,United-States,16th March 2025
3,service was outrageous,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,Chambery | London,January 2025,1.0,1.0,2.0,NaN,1.0,no,Trip Verified,This flight was a joke. There was four peop...,1,Luis De Jesus,United-States,16th March 2025
4,managed to get everything right,38 Paul Mercer | United-Kingdom | 7th March 2025,Boeing 777,Business,Premium Economy,Gatwick | Antigua,March 2025,5.0,5.0,4.0,4.0,5.0,yes,Trip Verified,This time British Airways managed to get ev...,4,38 Paul Mercer,United-Kingdom,7th March 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,welcome wasn’t especially friendly,K Pellberg | Sweden | 10th September 2019,Boeing 787 / A320,Solo Leisure,Business Class,Mahe | S|ckholm | London,September 2019,1.0,2.0,2.0,4.0,1.0,no,Trip Verified,Mahe to Stockholm via London. Checked in onli...,2,K Pellberg,Sweden,10th September 2019
996,been upgraded to Club World,John Allan | United-Kingdom | 8th September 2019,A319 / Boeing777,Couple Leisure,Business Class,Glasgow | Tampa | Gatwick,September 2019,4.0,5.0,3.0,3.0,5.0,yes,Trip Verified,Glasgow to Tampa via Gatwick. Early morning ...,1,John Allan,United-Kingdom,8th September 2019
997,standards on BA have plummeted,S Bailey | United-Kingdom | 7th September 2019,Boeing 777-200,Family Leisure,Economy Class,Orlando | Gatwick,September 2019,2.0,3.0,1.0,4.0,3.0,no,Not Verified,Orlando to Gatwick. Travelling home was extr...,2,S Bailey,United-Kingdom,7th September 2019
998,Good value flight,M Leyton | United Arab Emirates | 6th Septembe...,Boeing 787,Family Leisure,Economy Class,London | Abu Dhabi,August 2019,3.0,4.0,5.0,5.0,4.0,yes,Trip Verified,London to Abu Dhabi. This is the daytime fli...,4,M Leyton,United Arab Emirates,6th September 2019


In [178]:
results_df['Route']=results_df['Route'].str.replace(r'\ to ', '|', regex=True)
results_df['Route']=results_df['Route'].str.replace(r'via', '|', regex=True)
results_df

,Title,Details,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Recommended,Verified,Review,Sentiment,From,To,Via
0,disagreeable flight attendant,10 J Meares | United-Kingdom | 19th March 2025,Boeing 787,Solo Leisure,Business Class,Cape Town | London,March 2025,5.0,2.0,5.0,5.0,4.0,yes,Trip Verified,Flight mainly let down by a disagreeable fl...,2,10 J Meares,United-Kingdom,19th March 2025
1,flight was delayed nearly 1 hour,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,London | Geneva,December 2024,1.0,1.0,NaN,NaN,1.0,no,Trip Verified,Another awful experience by British Airways...,1,Luis De Jesus,United-States,16th March 2025
2,full of attitude to me,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Premium Economy,London | New York,January 2025,1.0,1.0,1.0,NaN,5.0,no,Trip Verified,"The service was rude, full of attitude to m...",2,Luis De Jesus,United-States,16th March 2025
3,service was outrageous,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,Chambery | London,January 2025,1.0,1.0,2.0,NaN,1.0,no,Trip Verified,This flight was a joke. There was four peop...,1,Luis De Jesus,United-States,16th March 2025
4,managed to get everything right,38 Paul Mercer | United-Kingdom | 7th March 2025,Boeing 777,Business,Premium Economy,Gatwick | Antigua,March 2025,5.0,5.0,4.0,4.0,5.0,yes,Trip Verified,This time British Airways managed to get ev...,4,38 Paul Mercer,United-Kingdom,7th March 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,welcome wasn’t especially friendly,K Pellberg | Sweden | 10th September 2019,Boeing 787 / A320,Solo Leisure,Business Class,Mahe | S|ckholm | London,September 2019,1.0,2.0,2.0,4.0,1.0,no,Trip Verified,Mahe to Stockholm via London. Checked in onli...,2,K Pellberg,Sweden,10th September 2019
996,been upgraded to Club World,John Allan | United-Kingdom | 8th September 2019,A319 / Boeing777,Couple Leisure,Business Class,Glasgow | Tampa | Gatwick,September 2019,4.0,5.0,3.0,3.0,5.0,yes,Trip Verified,Glasgow to Tampa via Gatwick. Early morning ...,1,John Allan,United-Kingdom,8th September 2019
997,standards on BA have plummeted,S Bailey | United-Kingdom | 7th September 2019,Boeing 777-200,Family Leisure,Economy Class,Orlando | Gatwick,September 2019,2.0,3.0,1.0,4.0,3.0,no,Not Verified,Orlando to Gatwick. Travelling home was extr...,2,S Bailey,United-Kingdom,7th September 2019
998,Good value flight,M Leyton | United Arab Emirates | 6th Septembe...,Boeing 787,Family Leisure,Economy Class,London | Abu Dhabi,August 2019,3.0,4.0,5.0,5.0,4.0,yes,Trip Verified,London to Abu Dhabi. This is the daytime fli...,4,M Leyton,United Arab Emirates,6th September 2019


In [166]:
results_df

,Title,Details,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Recommended,Verified,Review,Sentiment,From,To,Via
0,disagreeable flight attendant,10 J Meares | United-Kingdom | 19th March 2025,Boeing 787,Solo Leisure,Business Class,Cape Town | London,March 2025,5.0,2.0,5.0,5.0,4.0,yes,Trip Verified,Flight mainly let down by a disagreeable fl...,2,10 J Meares,United-Kingdom,19th March 2025
1,flight was delayed nearly 1 hour,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,London | Geneva,December 2024,1.0,1.0,NaN,NaN,1.0,no,Trip Verified,Another awful experience by British Airways...,1,Luis De Jesus,United-States,16th March 2025
2,full of attitude to me,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Premium Economy,London | New York,January 2025,1.0,1.0,1.0,NaN,5.0,no,Trip Verified,"The service was rude, full of attitude to m...",2,Luis De Jesus,United-States,16th March 2025
3,service was outrageous,Luis De Jesus | United-States | 16th March 2025,NaN,Business,Business Class,Chambery | London,January 2025,1.0,1.0,2.0,NaN,1.0,no,Trip Verified,This flight was a joke. There was four peop...,1,Luis De Jesus,United-States,16th March 2025
4,managed to get everything right,38 Paul Mercer | United-Kingdom | 7th March 2025,Boeing 777,Business,Premium Economy,Gatwick | Antigua,March 2025,5.0,5.0,4.0,4.0,5.0,yes,Trip Verified,This time British Airways managed to get ev...,4,38 Paul Mercer,United-Kingdom,7th March 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,welcome wasn’t especially friendly,K Pellberg | Sweden | 10th September 2019,Boeing 787 / A320,Solo Leisure,Business Class,Mahe | S|ckholm | London,September 2019,1.0,2.0,2.0,4.0,1.0,no,Trip Verified,Mahe to Stockholm via London. Checked in onli...,2,K Pellberg,Sweden,10th September 2019
996,been upgraded to Club World,John Allan | United-Kingdom | 8th September 2019,A319 / Boeing777,Couple Leisure,Business Class,Glasgow | Tampa | Gatwick,September 2019,4.0,5.0,3.0,3.0,5.0,yes,Trip Verified,Glasgow to Tampa via Gatwick. Early morning ...,1,John Allan,United-Kingdom,8th September 2019
997,standards on BA have plummeted,S Bailey | United-Kingdom | 7th September 2019,Boeing 777-200,Family Leisure,Economy Class,Orlando | Gatwick,September 2019,2.0,3.0,1.0,4.0,3.0,no,Not Verified,Orlando to Gatwick. Travelling home was extr...,2,S Bailey,United-Kingdom,7th September 2019
998,Good value flight,M Leyton | United Arab Emirates | 6th Septembe...,Boeing 787,Family Leisure,Economy Class,London | Abu Dhabi,August 2019,3.0,4.0,5.0,5.0,4.0,yes,Trip Verified,London to Abu Dhabi. This is the daytime fli...,4,M Leyton,United Arab Emirates,6th September 2019


In [168]:
results_df.to_csv("british_airways-2.csv", index=False)